# Lesson 03 EXTRA: SQL Query Examples + Data Visualization

## Database Applications Development

**Welcome to the bonus content!** 

This notebook is **optional** and designed for students who:
- Want more SQL query practice
- Are curious about data visualization
- Like exploring datasets
- Finished the main lesson early and want a challenge

### What's Different Here:
-  More interesting SQL queries
-  Cool charts and graphs (using matplotlib)
-  Deeper exploration of the Titanic data
-  No pressure - this is for fun and learning!

### About the Visualizations:
**You don't need to learn matplotlib right now.** We're just showing you what's possible! The code creates charts automatically. Later in the course, you might learn how to make your own.

**The Goal:** See how fast and powerful Python is for analyzing data and answering questions visually.

Let's dive in!

---

## Setup: Import Libraries and Load Database

First, let's get everything ready.

In [5]:
# Import libraries
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-darkgrid')   # Apply a pre-made style so all charts have a clean, dark grid look
# Make charts appear directly inside the notebook instead of opening a new window

%matplotlib inline  

print("Libraries imported successfully!")

Libraries imported successfully!


In [6]:
# Connect to the database (should already exist from main lesson)
conn = sqlite3.connect('titanic.db')

# Quick check: How many passengers?
count_check = pd.read_sql("SELECT COUNT(*) as total_passengers FROM passengers", conn)
print(f"Database connected: {count_check['total_passengers'][0]} passengers loaded")

DatabaseError: Execution failed on sql 'SELECT COUNT(*) as total_passengers FROM passengers': no such table: passengers

---

## Part 1: More Interesting SQL Queries

These queries go a bit deeper than the main lesson. Try to understand what each one does!

### Query 1: Find passengers by ticket class

Let's find all first-class passengers.

In [ ]:
# Find first-class passengers, sorted by highest fare
query = """
SELECT Name, Age, Pclass, Fare
FROM passengers
WHERE Pclass = 1
ORDER BY Fare DESC         -- Sort by fare, highest first
LIMIT 10                   -- Still only show the first 10 results
"""

result = pd.read_sql(query, conn)

print("Top 10 Most Expensive First-Class Passengers:")
result


In [ ]:
# Find first-class passengers
query = """
SELECT Name, Age, Pclass, Fare
FROM passengers
LIMIT 10
"""

result = pd.read_sql(query, conn)
print("First 10 First-Class Passengers:")
result

**New SQL keyword:** `WHERE`
- Filters rows based on a condition
- `WHERE Pclass = 1` means "only show rows where passenger class equals 1"
- Similar to pandas: `df[df['Pclass'] == 1]`

### Query 2: Find survivors

Who survived the disaster?

In [ ]:
# Find survivors
query = """
SELECT Name, Age, Sex, Pclass, Survived
FROM passengers
WHERE Survived = 1
LIMIT 15
"""

result = pd.read_sql(query, conn)
print(f"First 15 Survivors (out of {len(result)} total):")
result

**Note:** `Survived = 1` means the passenger survived. `Survived = 0` means they didn't.

### Query 3: Find young passengers

Who were the children on board?

In [ ]:
# Find passengers under 12 years old
query = """
SELECT Name, Age, Sex, Survived
FROM passengers
WHERE Age < 12
ORDER BY Age
LIMIT 20
"""

result = pd.read_sql(query, conn)
print("Youngest 20 passengers on the Titanic:")
result

**New SQL keywords:**
- `WHERE Age < 12` - only passengers under 12
- `ORDER BY Age` - sort by age (youngest first)
- Similar to pandas: `df[df['Age'] < 12].sort_values('Age')`

### Query 4: Multiple conditions

Find female first-class passengers.

In [ ]:
# Female first-class passengers
query = """
SELECT Name, Age, Sex, Pclass, Survived, Fare
FROM passengers
WHERE Sex = 'female' AND Pclass = 1
ORDER BY Fare DESC
LIMIT 10
"""

result = pd.read_sql(query, conn)
print("Female First-Class Passengers (highest fares):")
result

**New SQL keywords:**
- `AND` - combine multiple conditions (both must be true)
- `DESC` - descending order (highest to lowest)
- `ASC` would be ascending (lowest to highest)

---

## Part 2: Aggregation Queries

These queries **summarize** data - counting, averaging, finding min/max, etc.

### Count passengers by class

In [ ]:
# Count how many passengers in each class
query = """
SELECT Pclass, COUNT(*) as passenger_count
FROM passengers
GROUP BY Pclass
ORDER BY Pclass
"""

class_counts = pd.read_sql(query, conn)
print("Passengers by Class:")
class_counts

**New SQL keywords:**
- `GROUP BY` - group rows by a column
- `COUNT(*)` - count rows in each group
- `as passenger_count` - rename the result column

**Similar to pandas:** `df.groupby('Pclass').size()`

### Average age by class

In [ ]:
# Average age for each class
query = """
SELECT Pclass, 
       AVG(Age) as average_age,
       MIN(Age) as youngest,
       MAX(Age) as oldest
FROM passengers
WHERE Age IS NOT NULL
GROUP BY Pclass
ORDER BY Pclass
"""

age_stats = pd.read_sql(query, conn)
print("Age Statistics by Class:")
age_stats

**New SQL functions:**
- `AVG()` - average/mean
- `MIN()` - minimum value
- `MAX()` - maximum value
- `WHERE Age IS NOT NULL` - exclude rows where age is missing

### Survival rates by gender

In [ ]:
# Calculate survival rate for each gender
query = """
SELECT Sex,
       COUNT(*) as total,
       SUM(Survived) as survived,
       ROUND(AVG(Survived) * 100, 1) as survival_rate_percent
FROM passengers
GROUP BY Sex
"""

survival_by_gender = pd.read_sql(query, conn)
print("Survival Rates by Gender:")
survival_by_gender

**New SQL functions:**
- `SUM()` - add up values
- `ROUND(value, decimals)` - round to decimal places
- `AVG(Survived) * 100` - converts 0-1 to percentage

**Insight:** Women had a much higher survival rate! ("Women and children first" policy)

---

## Part 3: Data Visualization with Matplotlib

Now let's make the data come alive with charts! 📊

**Remember:** You don't need to understand all this code yet. Just enjoy the visuals and see what's possible!

### Visualization 1: Survival by Class

Did passenger class affect survival?

In [ ]:
# Query: Get survival counts by class
query = """
SELECT Pclass,
       SUM(CASE WHEN Survived = 1 THEN 1 ELSE 0 END) as survived,
       SUM(CASE WHEN Survived = 0 THEN 1 ELSE 0 END) as died
FROM passengers
GROUP BY Pclass
ORDER BY Pclass
"""

survival_by_class = pd.read_sql(query, conn)

# Create the chart
fig, ax = plt.subplots(figsize=(10, 6))

x = ['First Class', 'Second Class', 'Third Class']
survived = survival_by_class['survived']
died = survival_by_class['died']

width = 0.35
x_pos = range(len(x))

ax.bar([p - width/2 for p in x_pos], survived, width, label='Survived', color='#2ecc71', alpha=0.8)
ax.bar([p + width/2 for p in x_pos], died, width, label='Died', color='#e74c3c', alpha=0.8)

ax.set_xlabel('Passenger Class', fontsize=12)
ax.set_ylabel('Number of Passengers', fontsize=12)
ax.set_title('Titanic Survival by Passenger Class', fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(x)
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nWhat does this tell us?")
print("First-class passengers had the highest survival rate.")
print("Third-class passengers had the lowest survival rate.")

### Visualization 2: Age Distribution

How old were the passengers?

In [ ]:
# Query: Get all ages (excluding nulls)
query = """
SELECT Age
FROM passengers
WHERE Age IS NOT NULL
"""

ages = pd.read_sql(query, conn)

# Create histogram
fig, ax = plt.subplots(figsize=(10, 6))

ax.hist(ages['age'], bins=30, color='#3498db', alpha=0.7, edgecolor='black')
ax.set_xlabel('Age (years)', fontsize=12)
ax.set_ylabel('Number of Passengers', fontsize=12)
ax.set_title('Age Distribution of Titanic Passengers', fontsize=14, fontweight='bold')
ax.axvline(ages['age'].mean(), color='red', linestyle='--', linewidth=2, label=f'Average Age: {ages["age"].mean():.1f}')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n Age Statistics:")
print(f"   Average age: {ages['age'].mean():.1f} years")
print(f"   Youngest passenger: {ages['age'].min():.1f} years")
print(f"   Oldest passenger: {ages['age'].max():.1f} years")

### Visualization 3: Survival Rate by Gender

"Women and children first" - did this policy work?

In [ ]:
# Use the query we ran earlier
survival_by_gender

# Create bar chart
fig, ax = plt.subplots(figsize=(8, 6))

colors = ['#e91e63', '#2196f3']
ax.bar(survival_by_gender['sex'], survival_by_gender['survival_rate_percent'], 
       color=colors, alpha=0.8, edgecolor='black')

ax.set_xlabel('Gender', fontsize=12)
ax.set_ylabel('Survival Rate (%)', fontsize=12)
ax.set_title('Survival Rate by Gender', fontsize=14, fontweight='bold')
ax.set_ylim(0, 100)
ax.grid(axis='y', alpha=0.3)

# Add percentage labels on bars
for i, (sex, rate) in enumerate(zip(survival_by_gender['sex'], survival_by_gender['survival_rate_percent'])):
    ax.text(i, rate + 2, f'{rate}%', ha='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n Key Finding:")
print(f"   Female survival rate: {survival_by_gender.loc[survival_by_gender['sex']=='female', 'survival_rate_percent'].values[0]:.1f}%")
print(f"   Male survival rate: {survival_by_gender.loc[survival_by_gender['sex']=='male', 'survival_rate_percent'].values[0]:.1f}%")
print("   The 'women and children first' policy clearly had an effect!")

### Visualization 4: Fare Distribution by Class

How much did tickets cost?

In [ ]:
# Query: Get fares for each class
query = """
SELECT Pclass, Fare
FROM passengers
WHERE Fare IS NOT NULL AND Fare > 0
"""

fares = pd.read_sql(query, conn)

# Create box plot
fig, ax = plt.subplots(figsize=(10, 6))

class_1 = fares[fares['pclass'] == 1]['fare']
class_2 = fares[fares['pclass'] == 2]['fare']
class_3 = fares[fares['pclass'] == 3]['fare']

box_data = [class_1, class_2, class_3]
bp = ax.boxplot(box_data, labels=['First Class', 'Second Class', 'Third Class'],
                patch_artist=True, showmeans=True)

# Color the boxes
colors = ['#f39c12', '#9b59b6', '#16a085']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.set_xlabel('Passenger Class', fontsize=12)
ax.set_ylabel('Fare (British Pounds)', fontsize=12)
ax.set_title('Ticket Prices by Class', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n Fare Statistics:")
print(f"   First Class - Average: £{class_1.mean():.2f}, Max: £{class_1.max():.2f}")
print(f"   Second Class - Average: £{class_2.mean():.2f}, Max: £{class_2.max():.2f}")
print(f"   Third Class - Average: £{class_3.mean():.2f}, Max: £{class_3.max():.2f}")
print("\n   First-class tickets were MUCH more expensive!")

### Visualization 5: Survival by Age Group

Did age affect survival chances?

In [4]:
# Query: Categorize by age groups
query = """
SELECT 
    CASE 
        WHEN Age < 12 THEN 'Child (0-11)'
        WHEN Age < 18 THEN 'Teen (12-17)'
        WHEN Age < 35 THEN 'Young Adult (18-34)'
        WHEN Age < 60 THEN 'Adult (35-59)'
        ELSE 'Senior (60+)'
    END as age_group,
    AVG(Survived) * 100 as survival_rate
FROM passengers
WHERE Age IS NOT NULL
GROUP BY age_group
"""

age_survival = pd.read_sql(query, conn)

# Create bar chart
fig, ax = plt.subplots(figsize=(10, 6))

# Sort by age group logically
age_order = ['Child (0-11)', 'Teen (12-17)', 'Young Adult (18-34)', 'Adult (35-59)', 'Senior (60+)']
age_survival = age_survival.set_index('age_group').reindex(age_order).reset_index()

colors = ['#ff6b6b', '#ffd93d', '#6bcf7f', '#4d96ff', '#9d65c9']
bars = ax.bar(range(len(age_survival)), age_survival['survival_rate'], 
              color=colors, alpha=0.8, edgecolor='black')

ax.set_xlabel('Age Group', fontsize=12)
ax.set_ylabel('Survival Rate (%)', fontsize=12)
ax.set_title('Survival Rate by Age Group', fontsize=14, fontweight='bold')
ax.set_xticks(range(len(age_survival)))
ax.set_xticklabels(age_survival['age_group'], rotation=45, ha='right')
ax.set_ylim(0, 100)
ax.grid(axis='y', alpha=0.3)

# Add percentage labels
for i, rate in enumerate(age_survival['survival_rate']):
    ax.text(i, rate + 2, f'{rate:.1f}%', ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n Observation:")
print("   Children had the highest survival rate (women and children first!)")
print("   Adults had lower survival rates, especially males")

ProgrammingError: Cannot operate on a closed database.

---

## Cleanup

In [3]:
# Always close the connection when done!
conn.close()
print("Database connection closed")

Database connection closed


---

## What Did We Learn?

### New SQL Concepts:
- ✅ `WHERE` - filtering rows
- ✅ `ORDER BY` - sorting results
- ✅ `GROUP BY` - grouping data for summaries
- ✅ `AND` / `OR` - combining conditions
- ✅ Aggregate functions: `COUNT()`, `AVG()`, `SUM()`, `MIN()`, `MAX()`
- ✅ `CASE WHEN` - conditional logic in queries

### Data Visualization Power:
-  Bar charts show comparisons
-  Histograms show distributions
-  Box plots show ranges and outliers
-  Color and labels make data clear

### The Big Picture:
**Python + SQL + Matplotlib = Powerful Data Analysis!**

You can:
1. Query databases to get exactly the data you need
2. Analyze it with Python
3. Visualize it to see patterns and insights
4. Do all of this in seconds!

---

---

## Resources

**SQL Learning:**
- [W3Schools SQL Tutorial](https://www.w3schools.com/sql/)
- [SQLite Tutorial](https://www.sqlitetutorial.net/)

**Matplotlib Learning:**
- [Matplotlib Quick Start](https://matplotlib.org/stable/users/getting_started/)
- [Matplotlib Gallery](https://matplotlib.org/stable/gallery/index.html) - tons of examples!

**Remember:** You'll learn matplotlib properly later. For now, just enjoy seeing what it can do!

---
